# Computing and plotting monthly climatologies

Quick example of how to compute monthly climatologies of catalogued data and generate Hovmöller plots with the resulting averages. First, we start up a Kubernetes cluster and client:

In [4]:
from dask_kubernetes import KubeCluster

cluster = KubeCluster(n_workers=10)
cluster

In [5]:
from dask.distributed import Client

client = Client(cluster)
client

Client Scheduler: tcp://10.51.81.21:44993 Dashboard: /user/charlesbluca-pa-racmip-examples-8vouul33/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


Then we open up the preexisting catalog of TRACMIP data:

In [1]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/climate.yaml")
col = cat.tracmip()
col

pangeo-tracmip-ESM Collection with 7067 entries:
	> 3 frequency(s)

	> 11 experiment(s)

	> 14 model(s)

	> 47 variable(s)

	> 10 version(s)

	> 7067 source(s)

In [2]:
col.df

,frequency,experiment,model,variable,version,source
0,A3hr,aqua4xCO2,AM21,hur,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/hur/v20...
1,A3hr,aqua4xCO2,AM21,hus,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/hus/v20...
2,A3hr,aqua4xCO2,AM21,ta,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/ta/v201...
3,A3hr,aqua4xCO2,AM21,ua,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/ua/v201...
4,A3hr,aqua4xCO2,AM21,va,v20190116,gs://cmip6/tracmip/A3hr/aqua4xCO2/AM21/va/v201...
...,...,...,...,...,...,...
7062,Amon,landOrbit,MetUM-ENT,uas,v20180423,gs://cmip6/tracmip/Amon/landOrbit/MetUM-ENT/ua...
7063,Amon,landOrbit,MetUM-ENT,va,v20180423,gs://cmip6/tracmip/Amon/landOrbit/MetUM-ENT/va...
7064,Amon,landOrbit,MetUM-ENT,vas,v20180423,gs://cmip6/tracmip/Amon/landOrbit/MetUM-ENT/va...
7065,Amon,landOrbit,MetUM-ENT,wap,v20180423,gs://cmip6/tracmip/Amon/landOrbit/MetUM-ENT/wa...


In [6]:
ds_dict = col.search(frequency="Amon", experiment="aquaControl", 
                     variable="pr").to_dataset_dict()

C:\Users\geniu\Miniconda3\envs\pangeo-tracmip-examples\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'model.experiment.frequency'
             
--> There are 14 group(s)


Finally, we make a rough Hovmöller plot of this data, using `xarray`'s `groupby` method to produce monthly means:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(18, 20), facecolor='w', edgecolor='k')

for i, source in enumerate(ds_dict):
    ax = plt.subplot(5, 3, i + 1)
    ds_dict[source]["pr"].groupby("time.month").mean(dim=("time", "lon")) \
                         .plot.contourf(x="month", cmap="Blues",
                                        cbar_kwargs={"label" : ds_dict[source]["pr"].attrs["units"]})
    ax.set(title=ds_dict[source].attrs["model_id"])
    
plt.tight_layout()